In [54]:
import random
import string
import pandas as pand
import numpy as nump
import scipy as sci
import scipy.stats as stats
from matplotlib import pyplot

In [55]:
# Функции формирования 6 цифр ID клиента
def random_numbers():
       return ''.join(random.choice(string.digits) for x in range(6))

In [56]:
# Функция получения случайного ID клиента (по правилам в исходном задании)
def random_id():
       return ''.join(random.choice(string.ascii_uppercase) for x in range(3)) + random_numbers()

In [57]:
print(random_id())

VIL759858


In [58]:
# Формирование сырых данных (исходных по заданию) для увеличения выборки для последующего анализа
# Диапазон объема транзакций в рублях от 1р. до 600тыс.
N = 1000000 # Формирования записей 'транзакций' (ограничемся 1млн. строк)
numb = []
for i in range(N):
    numb.append(str(i+6)+','+random_id()+','+str(random.randint(1,600000))+','+random.choice(['R','AF']))

In [59]:
# Изначально имеется файл вида(по условию):
# 1,ADF346274,1000,R
# 2,JDY272800,600,AF
# 3,ADF346274,2460,R
# 4,TDG254776,4000,R
# 5,RYE846234,70000,AF
# Добавим заголовок "numb,ID,Vrub,Seg" в исходный файл для удобства дальнейшнего разбора нашего файла для анализа:
line_index = 0
lines = None
with open('transactions.txt', 'r') as file_handler:
    lines = file_handler.readlines()

lines.insert(line_index, "numb,ID,Vrub,Seg"+'\n')

with open('transactions.txt', 'w') as file_handler:
    file_handler.writelines(lines)

In [60]:
# Запишем в наш исходный файл, сгенерированные, соответственно правилам задания, данные:
with open(r"transactions.txt", "a") as file:
    file.write('\n')
    for line in numb:
        file.write(line + '\n')

In [97]:
data = pand.read_csv('transactions.txt', sep=',', encoding='utf-8-sig')
data.head(10)

,numb,ID,Vrub,Seg
0,1,ADF346274,1000,R
1,2,JDY272800,600,AF
2,3,ADF346274,2460,R
3,4,TDG254776,4000,R
4,5,RYE846234,70000,AF
5,6,EAM789233,560486,R
6,7,WOY783531,246724,R
7,8,ENA808770,6308,R
8,9,GCG946221,114011,R
9,10,XVZ754414,436073,AF


In [98]:
data.shape

(1000005, 4)

In [99]:
data.columns

Index(['numb', 'ID', 'Vrub', 'Seg'], dtype='object')

In [100]:
data.dtypes

numb     int64
ID      object
Vrub     int64
Seg     object
dtype: object

In [101]:
data.index

RangeIndex(start=0, stop=1000005, step=1)

In [102]:
data.describe()

,numb,Vrub
count,1.000005e+06,1.000005e+06
mean,5.000030e+05,3.000173e+05
std,2.886767e+05,1.732711e+05
min,1.000000e+00,1.000000e+00
25%,2.500020e+05,1.500500e+05
50%,5.000030e+05,2.997000e+05
75%,7.500040e+05,4.502240e+05
max,1.000005e+06,6.000000e+05


In [103]:
data.count()

numb    1000005
ID      1000005
Vrub    1000005
Seg     1000005
dtype: int64

In [105]:
# Подсчитает количество клиентов для каждого из сегментов, совершавших транзакции (исключает дубликаты по ID). 
data.groupby('Seg')['ID'].nunique()

Seg
AF    500053
R     499939
Name: ID, dtype: int64

In [106]:
# Подсчитает средний объем отдельной транзакции в каждом из сегментов. 
data.groupby('Seg').mean()[['Vrub']]

,Vrub
Seg,
AF,300082.657218
R,299951.940986


In [107]:
# Медиана
data.groupby('Seg').median()[['Vrub']]

,Vrub
Seg,
AF,299832.5
R,299547.0


In [108]:
sci.stats.poisson.ppf(0.9, data.groupby('Seg').mean()[['Vrub']])

array([[300785.],
       [300654.]])

In [109]:
data.groupby('Seg').describe()[['Vrub']]

Vrub                                                         \
        count           mean            std  min       25%       50%   
Seg                                                                    
AF   500058.0  300082.657218  173335.072890  1.0  149968.0  299832.5   
R    499947.0  299951.940986  173207.207125  1.0  150123.5  299547.0   

                         
          75%       max  
Seg                      
AF   450561.5  600000.0  
R    449848.0  600000.0

In [110]:
# Создадам массивы с данными об объеме транзакций для каждого из двух сегментов (для упрощения в дальнейшем анализе):
from array import *
SegmR = data['Seg'] == 'R'
SegmAF = data['Seg'] == 'AF'
VrubR = data['Vrub'] > 0
VrubAF = data['Vrub'] > 0
arrR = array('i', data.loc[VrubR & SegmR]['Vrub']) # Массив объема транзакций сегмента 'R'
arrAF = array('i', data.loc[VrubAF & SegmAF]['Vrub']) # Массив объема транзакций сегмента 'AF'

In [111]:
# Max & Min для двух сегментов
min_priceR = min(arrR)
max_priceR = max(arrR)
min_priceAF = min(arrAF)
max_priceAF = max(arrAF)
print(min_priceR, max_priceR)
print(min_priceAF, max_priceAF)

1 600000
1 600000


In [112]:
# Размах: max-min для двух сегментов
price_rangeR = max_priceR - min_priceR
print(price_rangeR)
price_rangeAF = max_priceAF - min_priceAF
print(price_rangeAF)

599999
599999


In [113]:
# Стандартные отклонения для двух сегментов:
print(nump.std(arrR))
print(nump.std(arrAF))

173207.03389914546
173334.89957525622


In [114]:
# Функция построения доверительного интервала (в 90%) для среднего объема транзакций в сегментах:
def dov_int(arr, alpha=0.9):
    n = len(arr)
    mu, se = nump.mean(arr), stats.sem(arr)
    bound = se * stats.t.ppf((1+alpha)/2., n-1)
    return mu-bound, mu+bound

In [115]:
# 90% доверительный интервал для среднего объема транзакций сегмента 'R'
ci = dov_int(arrR, alpha=0.9)
print(ci)

(299549.00872812414, 300354.8732433648)


In [116]:
# 90% доверительный интервал для среднего объема транзакций сегмента 'AF'
ci = dov_int(arrAF, alpha=0.9)
print(ci)

(299679.47226172354, 300485.8421738019)


In [117]:
# Строим t-статистику по двум выборкам (параметра 'объем транзакции (в рублях)' для каждого сегмента)
# Уровень значимости = 10% (по условию)
# Нулевая гипотеза - что средние объемы транзакций в каждом из двух сегментах равны.
stats.ttest_ind(arrR, arrAF)

Ttest_indResult(statistic=-0.3772022961980673, pvalue=0.7060233540434788)

In [ ]:
# Результат - По выведенному значению p-value, мы НЕ можем опровергнуть нулевую (искомую) гипотезу.